# 단어 표현
## Discrete Representation(==Local Representation)
값 그 자체를 매핑으로 표현하는 이산적인 표현 방법
### 희소표현(Sparse Representation)
One Hot Encoding 이 대표적인 예시로 단어 집합이 'Hello World' 두 개라고 할 때 인덱스로 표현하는 방법
$$Hello = [1, 0]$$
벡터가 대부분 0으로 차있기 때문에 One Hot Encoding은 Sparse Vector라고도 함
#### 장단점
* 단어집합이 많을 수록 1은 하나만 나타나고 나머지는 모두 0이라서 메모리 공간이 매우 낭비된다.  
* 대부분이 0이다 보니 유사도 계산같은 것 할 때 내적값이 0이 된다.

In [4]:
import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i need some more coffee more cake"
# 위 sentence
_input = [vocab[w] for w in sentence.split()]  # [0, 0, 0, 0, 1, 2, 3, 4, 4, 4]

vocab_size = len(vocab)   # 8

one_hot = tf.one_hot(_input, vocab_size)
print(one_hot.numpy(), one_hot.shape)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]] (7, 8)


### Count based
* Bag of Words (BoW)
* Document-Term Matrix (DTM)
* TDM
* Term Frequency-Inverse Document Frequency (TF - IDF)
* N-gram Language Model (N-gram)

## Continuous Representation
관계와 속성을 고려하여 실수로 표현

### 분산표현(Distributed Representation 또는 Dense Representation)
Sparse가 'syntax'를 표현하는 것이라면 'semantic'(의미) 위주로 표현해내는 것으로 각 벡터안에 실수들이 연속적으로 있고  
단어의 의미가 여러 차원에 분산되어 들어가 있기 때문에 Sparse <-> Dense 뺵뺵하게 모이게 된다.

**Sparse Representation**

![](https://www.oreilly.com/content/wp-content/uploads/sites/2/2020/01/Figure1-5eadbb96f1ac8c2b926b58d5a940f644.png)

**Dense Representation**

![](https://www.oreilly.com/content/wp-content/uploads/sites/2/2020/01/Figure2-9b757823f56b45a64820ab55a65918dd.png)

그래서 학습을 한다는 것은 N개의 단어를 쓰되 M개의 벡터로 표현하여 이 벡터에 담길 계수를 구하는 것이다.

#### 종류
* NNLM
* NPLM
* Word2Vec
* FastText
* Embedding from Language Model (ELMo)
* Bidirecional Language Model (biLM) 
### Count based
* Latent Semantic Analysis
### Distributed and count based
* GloVe


# Embedding


linear나 Embedding 레이어를 통해 학습할 수 있는 데이터로 변경할 수 있다. 아래 예제 코드에서는 단어를 2차원으로 분산하여 의미를 파악하도록 했다.  
따라서 나오는 결과 shape이 모두 7x2가 나오게 됬는데 7은 문장에서 사용된 단어의 수를, 2차원은 단어의 의미를 2차원으로 분산하여 해석함을 의미한다.  
레이어의 가중치 값을 찍어보면 단어가 one-hot 인코딩으로 되어 있어 단어가 있는 1에만 weight가 반응하였기 때문에 결과 레이어에서 중복되는 more의 값이 
Embedding Table에 'more'에 해당하는 인덱스 값임을 알 수 있다.

In [41]:
print(sentence)
print(one_hot)
print(vocab)

# 10차원으로 단어를 표현. 임베딩 테이블은 8 * 2이 됨
linear = tf.keras.layers.Dense(units=2, use_bias=False)
# 8개의 단어집단에 대해 각 단어를 2차원으로 표현. 임베딩 테이블은 8 * 2이 됨
embbeding = tf.keras.layers.Embedding(input_dim=8, output_dim=2)

print("\033[96;mLinear Layer\033[0;m")
linear_out = linear(one_hot)
print(linear.weights[0].shape, linear_out.shape)
print(linear.weights[0])
print(linear_out)

print("\033[96;mEmbedding Layer\033[0;m")
embbeding_out = embbeding(tf.constant([[0, 1, 2, 3, 4, 3, 5]]))
print(embbeding.weights[0].shape, embbeding_out.shape)
print(embbeding.weights[0])
print(embbeding_out)

i need some more coffee more cake
tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]], shape=(7, 8), dtype=float32)
{'i': 0, 'need': 1, 'some': 2, 'more': 3, 'coffee': 4, 'cake': 5, 'cat': 6, 'dog': 7}
Linear Layer
(8, 2) (7, 2)
<tf.Variable 'dense_36/kernel:0' shape=(8, 2) dtype=float32, numpy=
array([[-0.03398341,  0.6953498 ],
       [ 0.08553594, -0.7472702 ],
       [-0.28530037,  0.1955831 ],
       [ 0.29321814, -0.25269836],
       [ 0.19134903,  0.17799938],
       [-0.46871632, -0.10234278],
       [ 0.71975577, -0.6033121 ],
       [-0.23767686,  0.5163369 ]], dtype=float32)>
tf.Tensor(
[[-0.03398341  0.6953498 ]
 [ 0.08553594 -0.7472702 ]
 [-0.28530037  0.1955831 ]
 [ 0.29321814 -0.25269836]
 [ 0.19134903  0.17799938]
 [ 0.29321814 -0.25269836]
 [-0.46871632 -0.10234278]], shape=(7, 2), dtype=float32)
Embedding Layer
(8, 2) (1,

# Recurrent Neural Network
시계열 데이터(오디오, 주가, 문장) 등 요소의 순서가 중요한 데이터는 데이터의 전후관계가 매우 중요하여 이를 반영하여야 한다.
RNN은 이전 상태(t-1)에서 나온 결과를 현재 상태(t)에도 반영하는 방법이다. 이전에 사왔던 hidden state를 특정 계속 반영해주면 t=4일 때 이전 상태에서 쓰였던 값들이 함께 반영되어 가중치를 업데이트 하게 된다. 마치 IIR 필터와 비슷하다.  
![](https://miro.medium.com/v2/resize:fit:1400/1*xn5kA92_J5KLaKcP7BMRLA.gif)

각 변수에 대한 설명은 아래와 같다.
* t — time step
* X — input
* h — hidden state
* length of X — size/dimension of input
* length of h — no. of hidden units. Note that different libraries call them differently, but they mean the same:
  * Keras — state_size ,units
  * PyTorch — hidden_size
  * TensorFlow — num_units
  
![](https://miro.medium.com/v2/resize:fit:1400/1*1aOMOtRPXu4Ln2WtrwuM1A.gif)


In [78]:
encoded = tf.constant([[vocab[word] for word in sentence.split()]])
print(sentence, f"=> one-hot: {encoded}")

# 100차원으로 의미를 분산
layer_embedding = tf.keras.layers.Embedding(input_dim=len(vocab), output_dim=100)
embedded = layer_embedding(encoded)
print("Embedding Layer Table and distributed encoded sentence Shape", layer_embedding.weights[0].shape, embedded.shape)

layer_rnn1 = tf.keras.layers.SimpleRNN(units=10, return_sequences=True, use_bias=True)
layer_rnn1_out = layer_rnn1(embedded)
print("First RNN Layer weight and output shape:", layer_rnn1.weights[0].shape, layer_rnn1_out.shape)

from keras.utils import layer_utils
print(len(layer_rnn1.weights), layer_rnn1.weights[-1].shape)
print(f"trainable {layer_utils.count_params(layer_rnn1.trainable_weights)}")
print(f"trainable {layer_utils.count_params(layer_rnn1.non_trainable_weights)}")

i need some more coffee more cake => one-hot: [[0 1 2 3 4 3 5]]
Embedding Layer Table and distributed encoded sentence Shape (8, 100) (1, 7, 100)
First RNN Layer weight and output shape: (100, 10) (1, 7, 10)
3 (10,)
trainable 1110
trainable 0
tf.Tensor(
[[[-0.03876627  0.00804558 -0.00233553  0.00743397 -0.06500743
    0.00843441 -0.03965818  0.00280986 -0.02095365  0.07048263]
  [-0.04788142  0.02745524 -0.05689828 -0.08591075 -0.06910582
    0.01697299  0.04761899  0.05781495  0.01167249 -0.01159091]
  [-0.08142471 -0.0564367  -0.00728309 -0.08471774  0.07662702
   -0.02071115 -0.0232066   0.01804111 -0.04367734  0.06513503]
  [-0.00503353 -0.04515741 -0.0912676   0.08649882 -0.06090993
    0.00713351  0.03106842  0.06548575  0.04140548  0.11176973]
  [-0.05828673 -0.01038143 -0.04355701 -0.1454209   0.03573029
    0.03995705  0.15783969  0.10742193  0.03541525  0.01719273]
  [-0.11041305 -0.11941122 -0.04336985  0.03459192  0.07128581
    0.07869342  0.01101061 -0.05000282  0.015578

위 코드에서 layer_rnn1의 총 파라미터는 1100개고 1000개는 trainable, 100개는 non-trainable이다.  
$recurrent_weights + input_weights + biases$ 이 총 파라미터 갯수가 된다. 먼저 recurrent_weights와 input_weight를 계산해본다.  
weights[0] 에는 input_weight인 1000개가 들어있다. 100개의 차원으로 분석하였으니 feature는 100이 되고 아웃풋 뉴런이 10개이니 100 x 10 하여 1000개가 된다.  
그리고 recurrent_weights는 이전 시퀀스에서 가지고 온 아웃풋 뉴런이 10개이므로 이전 뉴런과 새로이 쓰일 뉴런을 곱해 10x10 100개 된다. 만야 bias또한 사용하였다면 아웃풋 뉴런에 붙을 것이므로 10 x 1이되서 1110개가 된다.

아래 예시는 입력이 3이고 중간 뉴런이 4개일 경우이다.
recurrent weight는 4*4으로 16개, input_weight는 4*3으로 12개, 바이어스는 4*1이되어 4개로 총 32개가 trainable parameter가 된다.
![](https://i.stack.imgur.com/71An9.png)

## 장단점
RNN의 경우 입력 데이터가 길면 길수록 concatenate를 해가면서 뒤로 전달할 때 앞단의 정보가 뒤로 갈수록 옅어지는 문제가 있다.
!()[https://research.aimultiple.com/wp-content/uploads/2021/08/rnn-text.gif]
### 기울기 소실과 기울기 폭발
역전파를 하면서 RNN역시 기울기 소실이 일어나는데 RNN은 기울기 폭발도 일어난다. 이전 상태로 역전파를 하면서 기울기를 구할 때 연쇄법칙에 의해 앞에 오는 미분계수값이 1보다 크게 되면 뒤로 점점 발산해버리는 현상이다. 두 경우 모두 가중치가 업데이트가 안되거나 아예 튀어버리므로 학습이 불가능하다.

